# 데이터 전처리

In [1]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

In [4]:
# 훈련 데이터 전처리
train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224,),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness= 0.2, contrast= 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 검증 데이터 전처리
eval_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224),
    transforms.CenterCrop(224),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  
])

# 데이터셋 로딩
train_dataset = datasets.ImageFolder('C:/DL/data/chest_xray/train', transform= train_transform)
val_dataset = datasets.ImageFolder('C:/DL/data/chest_xray/val', transform= eval_transform)
test_dataset = datasets.ImageFolder('C:/DL/data/chest_xray/test', transform= eval_transform)

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size= 32, shuffle= True)
val_loader = DataLoader(val_dataset, batch_size= 32)
test_loader = DataLoader(test_dataset, batch_size= 32)

# 모델 불러오기

In [ ]:
from efficientnet_pytorch import EfficientNet
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available else 'cpu'

# EfficientNet-b0 불러오기
model = EfficientNet.from_pretrained('efficientnet-b0')
model._fc = nn.Linear(model._fc.in_features, 2)
model = model.to(device)

Loaded pretrained weights for efficientnet-b0


# 학습 루프

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import GradScaler, autocast
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models import efficientnet_b0
import time

# 모델, 손실 함수, 옵티마이저 설정
model = efficientnet_b0(pretrained= True)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr= 1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode= min, factor= 0.5, patience= 3)